In [1]:
pip install pyltr

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pyltr

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)


In [3]:
metric = pyltr.metrics.NDCG(k=10)

In [4]:
pip install setuptools wheel numpy scipy scikit-learn -U

     |████████████████████████████████| 583 kB 672 kB/s eta 0:00:01
     |████████████████████████████████| 20.2 MB 5.1 MB/s eta 0:00:01     |████████████████████████████▍   | 17.9 MB 5.1 MB/s eta 0:00:01
Requirement already up-to-date: scipy in /usr/local/lib/python3.7/dist-packages (1.4.1)
     |████████████████████████████████| 6.8 MB 6.1 MB/s eta 0:00:01
ERROR: launchpadlib 1.10.6 requires testresources, which is not installed.
  Attempting uninstall: setuptools
    Found existing installation: setuptools 40.8.0
    Uninstalling setuptools-40.8.0:
      Successfully uninstalled setuptools-40.8.0
  Attempting uninstall: wheel
    Found existing installation: wheel 0.32.3
    Uninstalling wheel-0.32.3:
      Successfully uninstalled wheel-0.32.3
  Attempting uninstall: numpy
    Found existing installation: numpy 1.18.1
    Uninstalling numpy-1.18.1:
      Successfully uninstalled numpy-1.18.1
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.22.2.p

In [6]:
pip install lightgbm

     |████████████████████████████████| 1.2 MB 708 kB/s eta 0:00:01
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [109]:
import lightgbm as lgb
gbm = lgb.LGBMRanker()

In [110]:
import csv
import pandas as pd
data=pd.read_csv("train.csv")
#print(data)

In [112]:
from sklearn.decomposition import PCA
import numpy as np


In [113]:
data = data.loc[:, data.any()]
col=len(data.columns)
print(col)

42


In [114]:
ft=[str(i) for i in range(1,col+1) if str(i) in data.columns]
x_train=data[ft]
y_train=data["relevance"]
count=0
for val in y_train:
    if(val==1):
        count+=1
print(count)

8496


In [59]:
#PCA feature selection
# pca = PCA(n_components='mle',svd_solver = 'auto')
# pca.fit(x_train)
# print(pca.explained_variance_ratio_)
# print(pca.singular_values_)

[2.89337169e-01 1.72203266e-01 1.05344857e-01 8.55498745e-02
 7.69900456e-02 5.25775088e-02 3.17268075e-02 2.31301166e-02
 1.96916688e-02 1.66833788e-02 1.61035450e-02 1.48620261e-02
 1.37760939e-02 1.24893241e-02 1.21620798e-02 1.12241793e-02
 7.35996034e-03 6.68704270e-03 5.94889301e-03 5.83883054e-03
 5.05815475e-03 4.10478040e-03 2.69507584e-03 2.18763306e-03
 1.66728835e-03 1.40510222e-03 7.45199953e-04 6.02936939e-04
 3.89752624e-04 3.87091050e-04 3.19688159e-04 2.96292725e-04
 2.56180191e-04 1.19553582e-04 6.81126746e-05 8.49920304e-06]
[184.39095132 142.251935   111.26130748 100.26449696  95.11626304
  78.60270522  61.05913923  52.13464275  48.10372983  44.2771335
  43.50089947  41.79039744  40.23467877  38.30954187  37.80431869
  36.31740325  29.40865944  28.03202934  26.43964197  26.19391576
  24.37999134  21.96253439  17.79602413  16.03336687  13.99725418
  12.8496545    9.35780304   8.41731222   6.76755844   6.74441143
   6.12915729   5.90062459   5.48668798   3.74816574   

In [115]:
#Recursive Feature Elimination
from sklearn.feature_selection import RFE


In [116]:
print(x_train.shape[1])

37


In [117]:
data2=pd.read_csv("val.csv")
x_val=data2[ft]
y_val=data2["relevance"]

In [118]:
data3=pd.read_csv("test_x.csv")
x_test=data3[ft]

In [119]:
query_train = [x_train.shape[0]]
query_val = [x_val.shape[0]]
query_test = [x_test.shape[0]]

In [120]:
from sklearn.metrics import f1_score

In [121]:
gbm = lgb.LGBMClassifier(n_estimators=1000, min_split_gain=0.0,reg_lambda=0.8, silent=True,learning_rate=0.5,num_leaves=31,subsample_for_bin=200000,min_child_samples=20,min_child_weight=0.05,reg_alpha=0.5)


In [122]:
gbm.fit(x_train, y_train, eval_set=[(x_val, y_val)], early_stopping_rounds=5, verbose=False)

LGBMClassifier(learning_rate=0.5, min_child_weight=0.05, n_estimators=1000,
               reg_alpha=0.5, reg_lambda=0.8)

In [126]:
test_pred = gbm.predict(x_val)

In [127]:
scor=f1_score(y_val, test_pred, average='macro')
print(scor)

0.48167798388028876


In [128]:
test_pred = gbm.predict(x_test)
count=0
for val in test_pred:
    if(val==1):
        count+=1
print(count)

1147


In [32]:
import csv
with open('submission2.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["row_id","relevance"])
    i=0
    for val in test_pred:
        writer.writerow([i,val])
        i+=1
    

In [33]:
data4=pd.read_csv("submission2.csv")
print(data4)

       row_id  relevance
0           0          0
1           1          0
2           2          0
3           3          0
4           4          0
...       ...        ...
13921   13921          0
13922   13922          0
13923   13923          0
13924   13924          0
13925   13925          0

[13926 rows x 2 columns]
